In [ ]:
%matplotlib inline

Crop Labeled ImageData {#crop_labeled_example}
======================

Use `~pyvista.ImageDataFilters.crop`{.interpreted-text role="meth"} to
crop labeled data such as segmented medical images.


In [ ]:
from __future__ import annotations

import pyvista as pv
from pyvista import examples

Load a dataset with a CT image and corresponding segmentation labels.
Here we load
`~pyvista.examples.downloads.download_whole_body_ct_male`{.interpreted-text
role="func"}.


In [ ]:
dataset = examples.download_whole_body_ct_male()

Get the `~pyvista.ImageData`{.interpreted-text role="class"} for the CT
data and one of the segmentation masks. For this example we choose a
mask of the skull.


In [ ]:
ct = dataset['ct']
skull = dataset['segmentations']['skull']

Crop the CT image using the segmentation mask. Use `padding` to include
additional data points around the masked region.


In [ ]:
cropped_ct = ct.crop(mask=skull, padding=10)

Use `~pyvista.ImageDataFilters.points_to_cells`{.interpreted-text
role="meth"} to plot the cropped image as
`~pyvista.CellType.VOXEL`{.interpreted-text role="attr"} cells.


In [ ]:
cpos = [(687.5, 763.6, 471.3), (231.8, 296.3, 677.0), (0.107, 0.311, 0.944)]

cropped_ct_voxels = cropped_ct.points_to_cells()
cropped_ct_voxels.plot(volume=True, cpos=cpos)

Include a surface contour of the mask with the plot.


In [ ]:
skull_surface = skull.contour_labels()

pl = pv.Plotter()
pl.add_mesh(skull_surface, color='white')
pl.add_volume(cropped_ct_voxels)
pl.camera_position = cpos
pl.show()

After cropping, the CT image\'s dimensions are smaller than the mask\'s.


In [ ]:
cropped_ct.dimensions == skull.dimensions

To keep dimension the same, either

1.  crop the mask itself; the meshes will have smaller dimensions
    relative to the input
2.  pad the CT image as part of the initial crop; the meshes will have
    the same dimensions as the input

To crop the mask itself, you can perform a similar crop as before using
`mask=True`.


In [ ]:
cropped_skull = skull.crop(mask=True, padding=10)
cropped_skull.dimensions

However, computationally it is more efficient to crop using `extent`
directly.


In [ ]:
cropped_skull = skull.crop(extent=cropped_ct.extent)
cropped_skull.dimensions

Alternatively, use `keep_dimensions` and `fill_value` when initially
cropping the image so that the output dimensions match the input. A
value of `-1000` is used, which may represent air in the scan.


In [ ]:
cropped_ct = ct.crop(mask=skull, keep_dimensions=True, fill_value=-1000)
cropped_ct.dimensions